# Определение уязвимых групп населения
#### На основе данных о доходах, заболеваемости, социально-незащищенных слоях населения России и других экономических и демографических данных решаются задачи:
    • кластеризации регионов России и определение, какие из них наиболее остро нуждаются в помощи бедным/неблагополучным слоям населения 
    • описание групп населения, сталкивающихся с бедностью 
    • исследование вопросов:
        - влияет ли на уровень бедности в регионе число детей, пенсионеров и других социально-уязвимых групп?
        - связаны ли уровень бедности/социального неблагополучия с производством и потреблением в регионе?
        - какие еще зависимости можно пронаблюдать относительно социально незащищенных слоев населения? 
### 1 этап: загрузка данных из всех источников
#### Результат работы с отдельным источником - датафрейм готовый к объединению по индексу - названиям регионов. 

In [1]:
import pandas as pd

In [2]:
# Детская смертность 1
# добавим параметры
cmr_df = pd.read_excel('data/child_mortality_rural_1990_2021.xls', skiprows=2)

In [3]:
pd.set_option("max_rows", None)

In [4]:
cmr_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Российская Федерация,Оба пола,сельское население,11186.0,10943.0,10111.0,9840.0,9010.0,8582.0,7983.0,...,5463.0,5059.0,4681.0,4165.0,3568.0,3032.0,2521.0,2122.0,1916.0,1852.0
1,Центральный федеральный округ,Оба пола,сельское население,1615.0,1550.0,1361.0,1426.0,1347.0,1321.0,1200.0,...,625.0,608.0,487.0,490.0,426.0,347.0,291.0,211.0,228.0,212.0
2,Белгородская область,Оба пола,сельское население,103.0,92.0,75.0,79.0,80.0,72.0,72.0,...,43.0,48.0,41.0,42.0,36.0,34.0,33.0,16.0,22.0,20.0
3,Брянская область,Оба пола,сельское население,124.0,109.0,83.0,121.0,99.0,104.0,96.0,...,46.0,47.0,39.0,44.0,36.0,31.0,12.0,11.0,12.0,13.0
4,Владимирская область,Оба пола,сельское население,80.0,58.0,60.0,62.0,46.0,50.0,47.0,...,30.0,31.0,23.0,31.0,28.0,17.0,22.0,15.0,10.0,14.0


In [261]:
# удалим лишние колонки Оба пола и Селское население - везде
cmr_df.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)

In [262]:
cmr_df.rename(columns={'Unnamed: 0':"Region"}, inplace=True)

In [263]:
print("Уникальных значений:", cmr_df.Region.nunique())
cmr_df.loc[:, ['Region']].info()

Уникальных значений: 117
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Region  119 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB


In [264]:
def fix_regions(x):
    if pd.isnull(x):
        return x
    if x == 'nan':
        return None  
    if 'округ' in x and 'автономный' not in x:
        return None
    if 'Таймырский' in x:
        return None
    if 'федеральный округ' in x:
        return None
    if 'Федерация' in x:
        return None
    if 'район' in x:
        return None
    if 'Эвенкийский' in x:
        return None
    if 'Чеченская и Ингушская Республики' in x:
        return None    
    if 'Москва' in x:
        return 'Москва'
    if 'Санкт-Петербург' in x:
        return 'Санкт-Петербург'
    if 'Севастополь' in x:
        return 'Севастополь'
    if 'в том числе' in x:
        return None
    if 'Кемеровская' in x:
        return 'Кемеровская область - Кузбасс'
    if 'Чувашская' in x:
        return 'Чувашская Республика - Чувашия'
    
    x = x.replace('автономный округ', 'АО')
    
    if '(' in x:
        x = x[:x.find('(')] 
        
    return x

In [265]:
# применим условие для очистки названий регионов
cmr_df.Region = cmr_df.Region.apply(fix_regions)
# удалим лишние пробелы в названиях
cmr_df.Region = cmr_df['Region'].str.strip()
# убираем нерелевантные регионы
cmr_df = cmr_df.dropna(subset = ['Region'])
# назначим название области индексом
cmr_df.set_index('Region', inplace=True)

In [266]:
print("Уникальных значений:", cmr_df.index.nunique())

Уникальных значений: 85


In [267]:
# Детская смертность 2
# добавим параметры
cmu_df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/child_mortality_urban_1990_2021.xls', skiprows=2)

# удалим лишние колонки Оба пола и Селское население - везде
cmu_df.drop(['Unnamed: 1', 'Unnamed: 2'], axis = 1, inplace=True)
# переименуем для удобства колонку с названиями регионов
cmu_df.rename(columns={'Unnamed: 0':"Region"}, inplace=True)
# применим условие для очистки названий регионов
cmu_df.Region = cmu_df.Region.apply(fix_regions)
cmu_df = cmu_df.dropna(subset = ['Region'])
# удалим лишние пробелы в названиях
cmu_df.Region = cmu_df['Region'].str.strip()
# назначим название области индексом
cmu_df.set_index('Region', inplace=True)

print("Уникальных значений:", cmu_df.index.nunique())

Уникальных значений: 85


In [268]:
# валовый региональный продукт
grp_df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/gross_regional_product_1996_2020.xls', skiprows=2)

# переименуем для удобства колонку с названиями регионов
grp_df.rename(columns={'Unnamed: 0':"Region"}, inplace=True)
# применим условие для очистки названий регионов
grp_df.Region = grp_df.Region.apply(fix_regions)
grp_df = grp_df.dropna(subset = ['Region'])
# удалим лишние пробелы в названиях
grp_df.Region = grp_df['Region'].str.strip()
# назначим название области индексом
grp_df.set_index('Region', inplace=True)

print("Уникальных значений:", grp_df.index.nunique())

Уникальных значений: 85


In [269]:
set(cmr_df.index).difference(set(cmu_df.index))

set()

In [270]:
# денежные доходы населения
cri_df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/cash_real_income_wages_2015_2020.xlsx')

In [271]:
cri_df.head()

,region,2015,2016,2017,2018,2019,2020
0,Российская Федерация,302541.0,30865.0,31897.0,33266.0,35338.0,36073.0
1,Центральный\n федеральный округ,38832.0,40200.0,41897.0,44009.0,47252.0,48226.0
2,Белгородская область,28043.0,29799.0,30342.0,30778.0,32352.0,32841.0
3,Брянская область,23428.0,24006.0,25107.0,26585.0,28371.0,28596.0
4,Владимирская область,22712.0,22365.0,23554.0,23539.0,25358.0,25922.0


In [272]:
# денежные доходы населения
cri_df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/cash_real_income_wages_2015_2020.xlsx')

# переименуем для удобства колонку с названиями регионов
cri_df.rename(columns={'region':"Region"}, inplace=True)
# применим условие для очистки названий регионов
cri_df.Region = cri_df.Region.apply(fix_regions)
# удалим /n в названиях регионов
cri_df['Region'].replace(r'\s+|\\n', ' ', regex=True, inplace=True) 
cri_df = cri_df.dropna(subset = ['Region'])
# удалим лишние пробелы в названиях
cri_df.Region = cri_df['Region'].str.strip()
# назначим название области индексом
cri_df.set_index('Region', inplace=True)

print("Уникальных значений:", cri_df.index.nunique())

Уникальных значений: 85


In [273]:
set(cri_df.index).difference(set(cmr_df.index))

set()

In [189]:
# инвалидность - взять столбец всего и сгруппировать по годам
df = pd.read_csv('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/disabled_total_by_age_2017_2022.csv')
df.head()

,region,total,18_30,31_40,41_50,51_60,60_,date
0,Российская Федерация,11640873.0,550895.0,766054.0,1030652.0,2135436.0,7157836.0,2017-01-01
1,Центральный федеральный округ,3420310.0,118579.0,172662.0,257484.0,598102.0,2273483.0,2017-01-01
2,Белгородская область,223030.0,6318.0,10383.0,16596.0,37444.0,152289.0,2017-01-01
3,Брянская область,110418.0,4215.0,6568.0,10230.0,21481.0,67924.0,2017-01-01
4,Владимирская область,133352.0,4454.0,6811.0,9606.0,23322.0,89159.0,2017-01-01


In [ ]:
#домохозяйства - один показатель размер жилой площади 
df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/housing_2020.xlsx')
df.head()

In [ ]:
# Заболеваемость - Всего и несколько самых ключевых 

df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/morbidity_2005_2020_age_disease.xls', skiprows=2)

print(df['Unnamed: 1'].unique())
df.head()

In [ ]:
# население -  
df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/population.xlsx')
df.head(5)

In [ ]:
# малоимущее население по регионам всего
df = pd.read_csv('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/poverty_percent_by_regions_1992_2020.csv')
df.head()

In [ ]:
# малоимущее население по группам - даст три признака - малоимущие дети, старики и люди трудоспособного возраста
df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/poverty_socdem_2017.xls')
df.head()

In [ ]:
df = pd.read_csv('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/regional_production_2017_2020.csv')

print(df['production_field'].nunique()) # всего 133 различных категории - нужен творческий подход
df[140:160]

In [ ]:
# розничная торговля на душу населения рублей

df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/retail_turnover_per_capita_2000_2021.xls', skiprows=2)

df.head()

In [ ]:
# расходы на социальную политику

wes_df = pd.read_excel('C:/Users/Chichvarina Marina/Downloads/soc_rus_data/welfare_expense_share_2015_2020.xlsx')

# выявим строки содержащие федеральный округ, федерация и район
fo = wes_df[wes_df['region'].str.contains('федеральный округ|Федерация|район')].index
wes_df.drop(fo, axis = 0, inplace=True)

# назначим название области индексом
wes_df.rename(columns={'region':"Регион"}, inplace=True)
wes_df["Регион"] = wes_df["Регион"].str.strip()
wes_df.set_index('Регион', inplace=True)

print(wes_df.shape)
wes_df.head()

In [ ]:
# этап 2 объединение

In [ ]:
# среднее за 5 лет - детская смертность в сельской местности
cmr_mean = cmr_df.loc[:,'2016':'2020'].mean(axis=1)
cmr_mean

In [ ]:
# среднее за 5 лет - расходы на социальную политику в процентах
wes_mean = wes_df.loc[:,2016:2020].mean(axis=1)
wes_mean

In [ ]:
result = pd.concat([cmr_mean, wes_mean], axis=1).rename(columns={0:'Смертность,чел', 1:'Социальная политика,%'})
result

In [ ]:
set(result.index).difference(set(cmr_df.index))

In [ ]:
# для упрощения и ускорения процесса удалим регионы в которых остались пропуски
result.dropna(inplace=True)

In [ ]:
# этап 3 сравнение различных алгоритмов кластеризации, подбор параметров

In [ ]:
# импортируем библиотеки numpy и sklearn
import numpy as np
from sklearn.mixture import GaussianMixture

gm_clustering = GaussianMixture(n_components=4, random_state=42)

# обучаем модель 
gm_clustering.fit(result)

# для матрицы X получаем предсказания, к какому кластеру принадлежат объекты
gm_prediction = gm_clustering.predict(result)
gm_prediction

In [ ]:
gm_res = pd.Series(gm_prediction)
gm_res.index = result.index
result[gm_res==3]
# в третий кластер попал только один регион - при высоких расходах на социальную политику - высокая смертность

In [ ]:
result[gm_res==0] # нулевой кластер - низкая смертность

In [ ]:
result[gm_res==2]

In [ ]:
# этап 4 представление результатов, визуализация
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = 12, 12
plt.scatter(result['Смертность,чел'], result['Социальная политика,%'], c=gm_prediction, alpha=0.5, linewidths=7)
plt.show()

In [ ]:
# попробуем другой алгоритм и другое количество кластеров

from sklearn.cluster import KMeans

kmeans = KMeans(5, random_state=0)
labels = kmeans.fit(result).predict(result)

plt.scatter(result['Смертность,чел'], result['Социальная политика,%'], c=labels, alpha=0.5, linewidths=7);

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[63.391522, 96.328125], zoom_start=3)

In [ ]:
m

In [ ]:
# загружаем данные по индексам регионов
import json

with open('RF_admin/admin_level_4.geojson', encoding='utf-8') as json_file:
    map_data = json.load(json_file)

region_id = []
reg_list = map_data['features'][1:]
for el in reg_list:
    region_id.append((el['name'], el['id'])) 

region_id_df = pd.DataFrame(region_id)

region_id_df.columns = ['region', 'region_id']
region_id_df.head()

In [ ]:
region_id_df.rename(columns={'region':"Регион"}, inplace=True)
region_id_df["Регион"] = region_id_df["Регион"].str.strip()
region_id_df.set_index('Регион', inplace=True)
region_id_df.head()

In [ ]:
gm_res_df = pd.DataFrame()
gm_res_df['cluster'] = gm_res

In [ ]:
gm_res_df.head()

In [ ]:
result_map = region_id_df.join(gm_res_df, how='inner')
result_map

In [ ]:
rel_ = folium.Choropleth(
       geo_data = 'RF_admin/admin_level_4.geojson', 
       name = 'Кластеризация по регионам',
       data = result_map,
       columns=['region_id', 'cluster'], 
       key_on='id',
       bins = 4,
       fill_color='BuGn',
       nan_fill_color='darkblue',
       nan_fill_opacity=0.5,
       fill_opacity=0.7,
       line_opacity=0.2,
       legend_name= 'Кластеризация по регионам',
       highlight = True,
       show = False
)

In [ ]:
rel_.add_to(m);

In [ ]:
m